In [275]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, StackingClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import plot_roc_curve
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

### deal with some left over from preprocessing

In [297]:
df = pd.read_csv('step2_data/combined.csv', index_col=0)

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_57649/4094612143.py:1: DtypeWarning: Columns (21,47,51,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('step2_data/combined.csv', index_col=0)


In [298]:
df = df[df['RestaurantsPriceRange2'] != 'None']
df['RestaurantsPriceRange2'] = df['RestaurantsPriceRange2'].astype(float)
df = df.dropna(subset='RestaurantsPriceRange2')
df['RestaurantsPriceRange2'] = df['RestaurantsPriceRange2'].astype(int)

In [299]:
## True/False/None datatype issue: 'RestaurantsDelivery', 'BusinessAcceptsCreditCards', 'RestaurantsTakeOut', 'RestaurantsReservations', 'RestaurantsGoodForGroups'
true_false_col = ['RestaurantsDelivery', 'BusinessAcceptsCreditCards', 'RestaurantsTakeOut', 'RestaurantsReservations', 'RestaurantsGoodForGroups']

def tf_issue(col_li, df):

    for col in col_li:
        df[col] = df[col].astype('|S')
        df[col] = df[col].apply(lambda line: b'False' if line == b'nan' else line)
        df[col] = df[col].apply(lambda line: b'False' if line == b'None' else line)
        df[col] = LabelEncoder().fit_transform(df[col])
        df[col] = df[col].astype(int)

tf_issue(true_false_col, df)

In [300]:
# Weird u in the string
weird_u = ['WiFi', 'Alcohol', 'RestaurantsAttire', 'NoiseLevel', 'Smoking']

def u_issue(col_li, df):

    for col in col_li:
        df[col] = df[col].apply(lambda line: 'none' if type(line) == float else (line.split("\'")[1].lower() if len(line.split("\'")) == 3 else line.split("\'")[0].lower()))
        df[col] = LabelEncoder().fit_transform(df[col])
        df[col] = df[col].astype(int)

u_issue(weird_u, df)

In [301]:
# AcceptsInsurance, Open24Hours RestaurantsCounterService, has no useful value
# DietaryRestrictions have only 5 records
drop_col = ['business_id', 'AcceptsInsurance', 'Open24Hours', 'DietaryRestrictions', 'RestaurantsCounterService', 'Caters', 'HasTV', 'GoodForKids', 'DogsAllowed', 
            'HappyHour', 'WheelchairAccessible', 'OutdoorSeating', 'BikeParking', 'RestaurantsAttire', 'Ambience', 'Smoking', 'Music', 'GoodForDancing', 
            'BusinessAcceptsBitcoin', 'CoatCheck', 'BestNights', 'Corkage', 'BYOBCorkage', 'BYOB', 'AgesAllowed', 'ByAppointmentOnly', 'RestaurantsTableService',
            'DriveThru', 'BusinessParking', 'GoodForMeal']

df.drop(columns = drop_col, inplace=True)

In [302]:
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])
df['city'] = le.fit_transform(df['city'])

In [304]:
# delete columns only with one unique values
del_col = []

for i in df.columns:
    if df[i].nunique() <= 1:
        del_col.append(i)

df.drop(columns = del_col, inplace=True)

### Modeling

In [18]:
# prepare data for modeling
X = df.drop(columns='is_open')
y = df['is_open']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [ ]:
scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = xgb.XGBClassifier()